In [2]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
# Changer de répertoire (remplacez le chemin par celui de votre dossier)
os.chdir(r'D:\Data challenge')
print("Nouveau répertoire :", os.getcwd())

Nouveau répertoire : D:\Data challenge


In [6]:
import pandas as pd

# Charger le fichier Excel
file_path = "Time.xlsx"
df = pd.read_excel(file_path, sheet_name="Time")

# Convertir la date et créer la colonne "Début de semaine"
df['Work Date'] = pd.to_datetime(df['Work Date'], errors='coerce')
df = df.dropna(subset=['Work Date'])
df['Début de semaine'] = df['Work Date'] - pd.to_timedelta(df['Work Date'].dt.weekday, unit='D')

# Définir la stratégie d'agrégation
agg_dict = {
    col: 'sum' if pd.api.types.is_numeric_dtype(dtype) else 'first'
    for col, dtype in df.dtypes.items()
    if col not in ['Employee Name', 'Début de semaine']
}

# Agrégation par "Employee Name" et "Début de semaine"
grouped_df = df.groupby(['Employee Name', 'Début de semaine'], as_index=False).agg(agg_dict)

# Affichage (ou export CSV si besoin)
grouped_df.head()


,Employee Name,Début de semaine,Eng. No.,Eng. Description,Eng. Phase,Phase Description,Client No.,Client Name,Personnel No.,Staff Level,Work Date,Time Entry Date,Posting Date,Hours,Charge-Out Rate,Std. Price,Adm. Surcharge
0,Alice Dupont,2024-12-02 14:16:07,51032391869,Neptune,220,Mars,22000374506,Company X,319506,SPECIALIST/SENIOR CONSULT,2024-12-05 14:16:07,2024-12-06 14:16:07,2024-12-06 14:16:07,40.5,4620,8505,277.2
1,Alice Dupont,2024-12-09 14:16:07,41008580811,Neptune,140,Mars,14000238322,Company X,203322,SPECIALIST/SENIOR CONSULT,2024-12-11 14:16:07,2024-12-13 14:16:07,2024-12-13 14:16:07,32.5,2940,6825,176.4
2,Alice Dupont,2024-12-16 14:16:07,39011396857,Neptune,140,Mars,14000238322,Company X,203322,SPECIALIST/SENIOR CONSULT,2024-12-19 14:16:07,2024-12-20 14:16:07,2024-12-20 14:16:07,32.0,2940,6720,176.4
3,Alice Dupont,2025-01-06 14:16:07,21008322805,Neptune,80,Mars,8000136184,Company X,116184,SPECIALIST/SENIOR CONSULT,2025-01-10 14:16:07,2025-01-10 14:16:07,2025-01-10 14:16:07,40.0,1680,8400,100.8
4,Alice Dupont,2025-01-13 14:16:07,10004865414,Neptune,40,Mars,4000068092,Company X,58092,SPECIALIST/SENIOR CONSULT,2025-01-13 14:16:07,2025-01-17 14:16:07,2025-01-17 14:16:07,12.5,840,2625,50.4


In [12]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt


# 📆 Créer la colonne "Début de semaine"
df['Début de semaine'] = df['Work Date'] - pd.to_timedelta(df['Work Date'].dt.weekday, unit='D')

# 📊 Agrégation par employé et semaine
agg_dict = {
    col: 'sum' if pd.api.types.is_numeric_dtype(dtype) else 'first'
    for col, dtype in df.dtypes.items()
    if col not in ['Employee Name', 'Début de semaine']
}
grouped_df = df.groupby(['Employee Name', 'Début de semaine'], as_index=False).agg(agg_dict)

# 🧠 Préparation des features temporelles
weekly_avg = grouped_df.groupby('Début de semaine')['Hours'].mean().reset_index()
weekly_avg['week'] = weekly_avg['Début de semaine'].dt.isocalendar().week
weekly_avg['month'] = weekly_avg['Début de semaine'].dt.month
weekly_avg['year'] = weekly_avg['Début de semaine'].dt.year
weekly_avg['dayofyear'] = weekly_avg['Début de semaine'].dt.dayofyear
weekly_avg['sin_week'] = np.sin(2 * np.pi * weekly_avg['week'] / 52)
weekly_avg['cos_week'] = np.cos(2 * np.pi * weekly_avg['week'] / 52)

# 📌 Cible et variables
X = weekly_avg[['week', 'month', 'year', 'dayofyear', 'sin_week', 'cos_week']]
y = weekly_avg['Hours']

# ✂️ Séparer train/test (50/50 de manière temporelle)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False, random_state=42)

# 🌲 Modèle Random Forest
model = RandomForestRegressor( n_estimators=446,
    max_depth=16,
    min_samples_leaf=5,
    min_samples_split=7, 
    random_state=42)
model.fit(X_train, y_train)

# 📊 Validation croisée
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring="neg_root_mean_squared_error")
rmse_cv = -scores.mean()

# 🔮 Génération des 12 prochaines semaines
last_date = weekly_avg['Début de semaine'].max()
future_dates = [last_date + timedelta(weeks=i) for i in range(1, 13)]
future_df = pd.DataFrame({'Début de semaine': future_dates})
future_df['week'] = future_df['Début de semaine'].dt.isocalendar().week
future_df['month'] = future_df['Début de semaine'].dt.month
future_df['year'] = future_df['Début de semaine'].dt.year
future_df['dayofyear'] = future_df['Début de semaine'].dt.dayofyear
future_df['sin_week'] = np.sin(2 * np.pi * future_df['week'] / 52)
future_df['cos_week'] = np.cos(2 * np.pi * future_df['week'] / 52)

# 🔄 Prédictions
X_future = future_df[['week', 'month', 'year', 'dayofyear', 'sin_week', 'cos_week']]
future_df['Heures prévues'] = model.predict(X_future)

# 👥 Ajouter les noms des consultants pour une structure cohérente
consultants = grouped_df['Employee Name'].unique()
final_preds = pd.DataFrame([
    {"Employee Name": emp, "Début de semaine": row["Début de semaine"], "Heures prévues": row["Heures prévues"]}
    for emp in consultants
    for _, row in future_df.iterrows()
])

# Exporter le résultat
future_df.to_csv("predictions_par_consultant.csv", index=False)
print("📁 Fichier 'predictions_par_consultant.csv' exporté avec succès.")



📁 Fichier 'predictions_par_consultant.csv' exporté avec succès.


In [18]:
print("✅ Nouveau RMSE moyen (validation croisée sur train) :", -np.rmse_cv))


SyntaxError: unmatched ')' (333127013.py, line 1)

In [20]:
rmse_cv

9.65500089223185